In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.quantization import quantize_dynamic


def custom_quantize_dynamic(model, bits=8):
    quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
    scale_factor = 2 ** (8 - bits)
    for param in quantized_model.parameters():
        param.data = torch.round(param.data / scale_factor).to(torch.qint8)
    return quantized_model

GOOGLE COLAB에서 돌릴 경우 T4, L4 불가 A100 사용해야함.

모델 받는건 L4에서도 되는데 양자화에서 안 됨

매모리 터져서 죽어버림

In [ ]:
model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [3]:
quantized_model_8bit = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

양자화 연산이 Pytorch에서는 CUDA에서 지원되지 않기 때문에 CPU에서 실행(다음에 TensorRT로 해볼것)

In [6]:
device = torch.device("cpu")
model.to(device)
quantized_model_8bit.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): DynamicQuantizedLinear(in_features=4096, out_features=4096, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (k_proj): DynamicQuantizedLinear(in_features=4096, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (v_proj): DynamicQuantizedLinear(in_features=4096, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (o_proj): DynamicQuantizedLinear(in_features=4096, out_features=4096, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): DynamicQuantizedLinear(in_features=4096, out_features=14336, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (up_proj): DynamicQuantizedLinear(in_features=4096, ou

In [7]:
input_text = "안녕하세요, 만나서 반갑습니다. 무엇을 도와드릴?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

In [8]:
def measure_inference_time(model, inputs):
    start_time = time.time()
    with torch.no_grad():
        _ = model(**inputs)
    return time.time() - start_time

In [9]:
import time

original_model_time = measure_inference_time(model, inputs)
quantized_model_8bit_time = measure_inference_time(quantized_model_8bit, inputs)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [10]:
print(f"Original Model Inference Time: {original_model_time:.4f} seconds")
print(f"Quantized Model 8-bit Inference Time: {quantized_model_8bit_time:.4f} seconds")

Original Model Inference Time: 2.8371 seconds
Quantized Model (8-bit) Inference Time: 0.3740 seconds


### 결과
Original Model Inference Time: **2.8371** seconds
Quantized Model (8-bit) Inference Time: **0.3740** seconds